---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 9, 2024<br>
---

# Usefull function for the daily loads

In [0]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import matplotlib.ticker as mtick
from sklearn.preprocessing import LabelEncoder
import scipy.stats as ss
from math import pi

from scipy.optimize import minimize, Bounds, LinearConstraint

In [0]:
# Setting the matplotlib format to 'retina' for better display quality
set_matplotlib_formats('retina')
plt.style.use('seaborn-notebook')

In [0]:
#-----------------------------------------
# Garch(1, 1) Model
#-----------------------------------------
def garch(theta, r):
    """
    Compute the GARCH(1,1) volatility using the given parameters.

    Parameters:
    - theta (list): List containing GARCH parameters in the following order:
        - w (float): Constant term.
        - a (float): Coefficient for the lagged squared return term.
        - b (float): Coefficient for the lagged volatility term.
    - r (numpy.ndarray): Array of financial returns.

    Returns:
    - numpy.ndarray: Array containing the GARCH(1,1) volatility estimates for each period.
    """
    # Extract the parameters
    w = theta[0]
    a = theta[1]
    b = theta[2]

    # Initialize array to store volatility estimates
    T = len(r)
    sigma = np.zeros(T + 1)
    sigma[0] = w / (1 - a - b)

    # Iterate to calculate volatility estimates
    for i in range(1, T + 1):
        sigma[i] = w + a * (r[i - 1] ** 2) + b * sigma[i - 1]

    return sigma

In [0]:
#-----------------------------------------
# Normal distribution to minimize
#-----------------------------------------
def log_normal_likely(theta, r):
    """
    Calculate the log-likelihood of a GARCH(1,1) model assuming normally distributed returns.

    Parameters:
    - theta (list): List containing GARCH parameters in the following order:
        - w (float): Constant term.
        - a (float): Coefficient for the lagged squared return term.
        - b (float): Coefficient for the lagged volatility term.
    - r (numpy.ndarray): Array of financial returns.

    Returns:
    - float: Log-likelihood value for the given GARCH parameters and returns.
    """
    # Calculate GARCH(1,1) volatility using the provided parameters
    sigma = garch(theta, r)[:-1]

    # Calculate log-likelihood
    ll = (-np.log(np.sqrt(2 * pi * sigma)) - 0.5 * (r ** 2) / sigma).sum()

    # Check for finite log-likelihood values
    if np.isfinite(ll):
        return ll
    else:
        return 1e10

In [0]:
#-----------------------------------------
# Forecast the volatility for the next day
#-----------------------------------------
def forecast_vol(r):
    """
    Forecast the volatility using a GARCH(1,1) model based on historical financial returns.

    Parameters:
    - r (numpy.ndarray): Array of historical financial returns.

    Returns:
    - numpy.ndarray: Array containing the forecasted GARCH(1,1) volatility for each period.
    """
    # Initial guess for GARCH parameters
    initial_guess = [0.00001, 0.1, 0.8]

    # Define negative log-likelihood function
    neg_log_likelihood = lambda theta: -log_normal_likely(theta, r)

    # Define parameter bounds
    bounds = [(0, 1), (0.001, 0.999), (0.001, 0.999)]

    # Use the minimize function with the constraints
    result = minimize(neg_log_likelihood, initial_guess, method='BFGS', bounds=bounds)

    # Extract the optimal parameters
    optimal_theta = result.x

    sigma = garch(optimal_theta, r)

    return sigma, optimal_theta

In [0]:
#-----------------------------------------
# Simulate for retunr for the next t day
#-----------------------------------------
def forecast_return(r, t_days, n_sim):
    """
    Generates Monte Carlo simulations of future returns based on a GARCH(1,1) model.

    Parameters:
    - r (pd.Series): Time series of historical returns.
    - t_days (int): Number of trading days to simulate into the future.
    - n_sim (int): Number of Monte Carlo simulations to perform.

    Returns:
    - monte_carlo (list): List of simulated return sequences for each simulation.

    The function uses a GARCH(1,1) model to simulate future returns based on the given historical returns.
    It calculates the GARCH(1,1) parameters (sigma and theta) using the `forecast_vol` function.
    The Monte Carlo simulations are performed by generating random normal variables and applying the GARCH(1,1) model.

    Parameters:
    - r: Time series of historical returns.
    - t_days: Number of trading days to simulate into the future.
    - n_sim: Number of Monte Carlo simulations to perform.

    Returns:
    - monte_carlo: List of simulated return sequences for each simulation.
    """

    # Calculate GARCH(1,1) volatility parameter (theta) only once
    sig, theta = forecast_vol(r)

    # Initialize lists to store simulated returns and Monte Carlo simulations
    return_list_t_days = [0]
    monte_carlo = []

    # Perform Monte Carlo simulations
    for i in range(0, n_sim):
        # Generate a sequence of normal random variables
        norm = np.random.normal(0, 1, t_days)

        # Initialize temporary lists for each simulation
        sig_temp = [sig[-1]]
        ret_temp = [0, np.sqrt(sig[-1]) * norm[0]]

        # Simulate future returns using GARCH(1,1) model
        for j in range(0, t_days):
            sigma = theta[0] + theta[1]*ret_temp[j+1]**2 + theta[2]*sig_temp[j]
            sig_temp.append(sigma)
            r_t = norm[j] * np.sqrt(sigma)
            ret_temp.append(r_t)
            
        # Store simulated returns for each simulation
        monte_carlo.append(ret_temp)

    return monte_carlo